# Intro

The goal of this project is to develop a **fake news filtering software**.
To make such **filters**, I **utilized known features** of fake news as well as **newly identified features** of fake news from the **explorative data analysis with natural language processings**.

# Data

The dataset is provide by Information Security and Object Technology (ISOT) research lab, University of Victoria.
https://www.uvic.ca/engineering/ece/isot/datasets/fake-news/index.php

The dataset contains 21k of real news scrapped from "Reuters.com" and 24k of fake news collected from different sources, where all of them are flagged as unreliable by Polififact (a fact-checking organization in the USA) and Wikipedia. The coverage of topics are various, yet mostly about politics.

Provided dataset are **True.csv** (reuter news) and **Fake.csv** (unreliable news), both have **title, text, subject, and publication date**.

# Known characteristics of fake news

I summarized the judging criteria about credibility of news into three categories.

1. Information-wise

    1. Not a whole information
        
        1. Lack of necessary information, like 5W1H
        2. Lack of context

    2. Not a NEWs
    
        1. Outdated
        
    3. Not valuable
    
        1. Not impactful/important socially
        2. Not a rare event
        3. Nothing to do with the area where news provider cover
        
2. Tone

    1. Doesn't sounds professional
    
        1. Contain slangs
        2. Vocabularies are not specific
    
    2. Hateful
    
        1. Enhance bias or discrimination
        2. Provocative
        
    3. Urgent
    
        1. Make readers to spread this news as much as you can
        2. Make reader to act promptly
        
    4. Joke (or pretend to be a joke)
    
        1. Make fun of someone/organization/policy
        
    5. Clickbait

        1. The title contains the above

3. Source-wise

    1. Author
    
        1. Cannot find the name of author
        2. The author is fake
        3. The author is not a reliable person/organization
        
    2. Media/Publishing organization
    
        1. The media is not reliable of fishy

    3. Supporting evidence
    
        1. The evidence that support the news is not adequate
        2. Not provided by a relavant expert or organization

# EDA

The goal of this EDA is to find extractable and useful features which can utilize the known characterastics of fake news, as well as to find unexpected features of fake news.

In [10]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag, RegexpParser
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB 

In [11]:
df0 = pd.read_csv('True.csv')
df1 = pd.read_csv('Fake.csv')

In [16]:
print(df0.info())
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
None


### Observation 1
- Unfortunately, this dataset doesn't contain the name of author.
- There's no n/a entry.

In [26]:
pd.options.display.max_colwidth = 200
print(df0.head(20))
print(df1.head(20))

                                                                             title  \
0                 As U.S. budget fight looms, Republicans flip their fiscal script   
1                 U.S. military to accept transgender recruits on Monday: Pentagon   
2                     Senior U.S. Republican senator: 'Let Mr. Mueller do his job'   
3                      FBI Russia probe helped by Australian diplomat tip-off: NYT   
4            Trump wants Postal Service to charge 'much more' for Amazon shipments   
5                 White House, Congress prepare for talks on spending, immigration   
6                  Trump says Russia probe will be fair, but timeline unclear: NYT   
7                     Factbox: Trump on Twitter (Dec 29) - Approval rating, Amazon   
8                                       Trump on Twitter (Dec 28) - Global Warming   
9     Alabama official to certify Senator-elect Jones today despite challenge: CNN   
10                      Jones certified U.S. Senate wi

### Observation 2
- 
- "Subject" column doesn't seem to be useful

In [9]:


df0.iloc[4][1]
print(text)

SEATTLE/WASHINGTON (Reuters) - President Donald Trump called on the U.S. Postal Service on Friday to charge “much more” to ship packages for Amazon (AMZN.O), picking another fight with an online retail giant he has criticized in the past.     “Why is the United States Post Office, which is losing many billions of dollars a year, while charging Amazon and others so little to deliver their packages, making Amazon richer and the Post Office dumber and poorer? Should be charging MUCH MORE!” Trump wrote on Twitter.  The president’s tweet drew fresh attention to the fragile finances of the Postal Service at a time when tens of millions of parcels have just been shipped all over the country for the holiday season.  The U.S. Postal Service, which runs at a big loss, is an independent agency within the federal government and does not receive tax dollars for operating expenses, according to its website.  Package delivery has become an increasingly important part of its business as the Internet h

10% of articles don't have author. I wonder if that can be characteristics of fake news.

In [6]:
df_read.fillna({'author':'_EmptyAuthorTitleText_','title':'_EmptyAuthorTitleText_','text':'_EmptyAuthorTitleText_'},inplace=True)

In [7]:
df_read[df_read.author == '_EmptyAuthorTitleText_'].describe()

,id,label
count,1957.000000,1957.000000
mean,10292.462954,0.986714
std,6029.266152,0.114524
min,6.000000,0.000000
25%,5124.000000,1.000000
50%,10242.000000,1.000000
75%,15458.000000,1.000000
max,20786.000000,1.000000


100% of news that has no title or text were all fake, and 99% of news that cannot specify author are fake. They can be strong features with high precision.

In [22]:
vectorizer = CountVectorizer()

vectorizer.fit(text)

X = vectorizer.transform(text)
y = label

print(vectorizer.vocabulary_)

{'house': 3606, 'dem': 2059, 'aide': 422, 'we': 7929, 'didn': 2180, 'even': 2683, 'see': 6499, 'comey': 1572, 'letter': 4289, 'until': 7673, 'jason': 3999, 'chaffetz': 1365, 'tweeted': 7547, 'it': 3976, 'by': 1223, 'darrell': 1962, 'lucus': 4426, 'on': 5098, 'october': 5055, '30': 114, '2016': 91, 'subscribe': 7060, 'the': 7304, 'stump': 7048, 'in': 3732, 'american': 521, 'fork': 3048, 'utah': 7715, 'image': 3696, 'courtesy': 1847, 'michael': 4654, 'jolley': 4047, 'available': 801, 'under': 7592, 'creative': 1868, 'commons': 1597, 'license': 4312, 'with': 8026, 'apologies': 604, 'to': 7385, 'keith': 4097, 'olbermann': 5091, 'there': 7317, 'is': 3956, 'no': 4965, 'doubt': 2334, 'who': 7981, 'worst': 8077, 'person': 5393, 'world': 8065, 'this': 7332, 'week': 7946, 'fbi': 2869, 'director': 2211, 'james': 3993, 'but': 1214, 'according': 271, 'democratic': 2066, 'looks': 4398, 'like': 4328, 'also': 496, 'know': 4137, 'second': 6487, 'as': 691, 'well': 7955, 'turns': 7542, 'out': 5184, 'that

In [23]:
classifier = MultinomialNB()
classifier.fit(X,y)

MultinomialNB()

In [27]:
x = df_read.iloc[100]

print(x)

classifier.predict_proba(vectorizer.transform([x.content]))


id                                                       100
title      Technocracy: The Real Reason Why The UN Wants ...
author                                         Activist Post
text       By Patrick Wood By its very nature, the Intern...
label                                                      1
content    By Patrick Wood By its very nature, the Intern...
Name: 100, dtype: object


array([[0.01342701, 0.98657299]])

In [ ]:
# Get a random text
text = df.iloc[6][3]

# Sentence tokenize
text = sent_tokenize(text)

# Part of speech tagging
pos_text = []

# Stop word removal
stop_words = set(stopwords.words('english'))

t = ['this is good', 'that is good', 'good hehe', 'this is bad', 'that is bad', 'bad sucks']
l = [0, 0, 0, 1, 1, 1]


counter = CountVectorizer()
counter.fit(t)


print(counter.vocabulary_)
print(counter.transform(t))



After playing with the 'chunk_counter' function for a few sentences, I noticed that real news have more specific (you can guess topic based on keywords) and objective terms whereas fake news have more subjective and plain (no idea what's topic based on frequent keywords) terms.

In [ ]:


for sentence in text:
    
    sentence = re.sub('\W+',' ',sentence)
    sentence = sentence.lower()
    
    sentence = word_tokenize(sentence)
    
    keywords = [word for word in sentence if word not in stop_words]
    
    keywords = pos_tag(keywords)
    pos_text.append(keywords)
    
    

def chunk_counter(pos_text, abbr='NP', n_chunk=30):

    grammar = ''
    
    if abbr == 'Noun':
        grammar = "Noun: {<NN.*>}"
        
    elif abbr == 'Verb':
        grammar = "Verb: {<VB.*>}"
    
    elif abbr == 'Adj':
        grammar = 'Adj: {<JJ.*>}'
        
    elif abbr == 'Adv':
        grammar = 'Adv: {<RB.*>}'    
        
    #grammar = "NP: {<DT>?<JJ>*<NN.*>}" # noun
    #grammar = "VPa: {<DT>?<JJ>*<NN.*><VB.*><RB.?>?}"
    #grammar = "VPb: {<VB.*><DT>?<JJ>*<NN><RB.?>?}" 
    
    else:
        print('Incorrect gabbr')
        return False

        
    # Chunk phrases
    parser = RegexpParser(grammar)
    
    chunks = []
    
    for sentence in pos_text:
        
        chunk = parser.parse(sentence)
        
        for subtree in chunk.subtrees(filter=lambda t: t.label() == abbr):
            chunks.append(tuple(subtree))
        
    # Count phrases
    counter = Counter()

    for chunk in chunks:
        counter[chunk] += 1

    return counter.most_common(10)

print(chunk_counter(pos_text, abbr='Verb'))